<a href="https://colab.research.google.com/github/suhaaskarthik/amazon-anamolies/blob/main/anamoly_detection_for_amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip /content/drive/MyDrive/terrascope_download_20250605_192022.zip

In [2]:
!pip install openai rasterio numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 64.6 MB/s eta 0:00:00


In [3]:
import rasterio
import numpy as np
from rasterio.transform import xy
import os
from scipy.ndimage import binary_dilation
from sklearn.cluster import DBSCAN

In [ ]:
raster_paths = []
for tile in os.listdir('WORLDCOVER/ESA_WORLDCOVER_10M_2021_V200/MAP'):
  raster_paths.append('WORLDCOVER/ESA_WORLDCOVER_10M_2021_V200/MAP'+'/'+tile+'/'+tile+'.tif')

cluster_coord = []
anamolies = []
for raster_path in raster_paths:
  print(raster_path.split('/')[-1].split('_')[-2])
  forest_class = 30
  target_classes = [40]  # cropland, built-up, bare soil
  max_points = 5

  # === LOAD RASTER ===
  with rasterio.open(raster_path) as src:
      data = src.read(1)
      transform = src.transform
      crs = src.crs
  print(data.shape)
  # === FIND ANOMALIES ===
  forest_mask = (data == 10)
  anomaly_mask = np.isin(data, target_classes)

  # Dilate forest to simulate neighborhood
  forest_neighborhood = binary_dilation(forest_mask, iterations=1)

  # Keep anomaly pixels that are near or inside forest
  anomalies_near_forest = anomaly_mask & forest_neighborhood
  # === EXTRACT COORDINATES ===
  rows, cols = np.where(anomalies_near_forest)
  coords = [xy(transform, r, c) for r, c in zip(rows, cols)]
  print(f"Found {len(coords)} anomalies. Sending {max_points} to GPT.")
  X = np.array(coords)
  if len(X.shape)==1:
    continue
  # Step 3: Run DBSCAN clustering
  # eps is approx distance in degrees: ~0.001° ≈ 100 meters
  clustering = DBSCAN(eps=0.0015, min_samples=5).fit(X)

  # Step 4: Get unique clusters
  labels = clustering.labels_
  n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
  print(f"Found {n_clusters} anomaly clusters.")
  if n_clusters>100:
    continue
  # Step 5: Print cluster centers
  for cluster_id in range(n_clusters):
      cluster_points = X[labels == cluster_id]
      print(cluster_points)
      if len(cluster_points)<20 or len(cluster_points)>100:
        continue
      centroid = cluster_points.mean(axis=0)
      print(f"Cluster {cluster_id+1}: ~{len(cluster_points)} pixels, center = lat {centroid[1]:.5f}, lon {centroid[0]:.5f}")
      cluster_coord.append([centroid[1], centroid[0], len(cluster_points), cluster_points])

In [ ]:
len(cluster_coord)

129

In [4]:
import pickle
with open("anomalies.pkl", "rb") as f:
    cluster_coord = pickle.load(f)

In [ ]:
!pip install earthaccess

In [ ]:
import earthaccess
import os

# Step 1: Authenticate (will prompt a login in browser)
# Ensure you have successfully authenticated before running this cell.
earthaccess.login() # Uncomment and run this in a previous cell if you haven't logged in

# Step 2: Define your coordinates (e.g., Cluster 82)
latitude = cluster_coord[100][0]
longitude = cluster_coord[100][1]
print(latitude,longitude)
# Step 3: Create a bounding box: [west, south, east, north]
buffer_deg = 0.005  # ~500 meters
bbox = [
    longitude - buffer_deg,  # west
    latitude - buffer_deg,   # south
    longitude + buffer_deg,  # east
    latitude + buffer_deg    # north
]

# Step 4: Search for GEDI L2B data within that box and a date range
results = earthaccess.search_data(
    short_name="GEDI02_B",
    # Pass the bbox list directly to the bounding_box parameter
    bounding_box=(bbox[0], bbox[1], bbox[2], bbox[3]),
    temporal=("2021-01-01", "2021-12-31"),
    cloud_hosted=True
)

# Step 5: Download first file (or more)
if results:
    print(f"✅ Found {len(results)} file(s). Downloading the first one...")
    files = earthaccess.download(results[:1])
    print("📁 Saved to:", files[0])
else:
    print("❌ No GEDI data found for that coordinate and time.")


In [ ]:
!pip install h5py

In [ ]:
import h5py

# Replace with your downloaded file path
file_path = files[0]
f = h5py.File(file_path, "r")
beam = f['BEAM0000']

In [ ]:
quality = beam['l2b_quality_flag'][:]
good = quality == 1

# === Extract datasets ===
lat = beam['geolocation/lat_lowestmode'][:][good]
lon = beam['geolocation/lon_lowestmode'][:][good]
rh100 = beam['rh100'][:][good]  # 100th percentile canopy height
elev = beam['geolocation/elev_lowestmode'][:][good]  # Ground elevation
shot_id = beam['shot_number'][:][good]

# === Preview first few good-quality shots ===
print(f"✅ Found {len(lat)} high-quality GEDI shots.\n")
for i in range(min(5, len(lat))):
    print(f"Shot {shot_id[i]}:")
    print(f"  Latitude       : {lat[i]:.6f}")
    print(f"  Longitude      : {lon[i]:.6f}")
    print(f"  RH100 (canopy) : {rh100[i]:.2f} m")
    print(f"  Elevation      : {elev[i]:.2f} m\n")

✅ Found 18731 high-quality GEDI shots.

Shot 124160200200076114:
  Latitude       : -0.080366
  Longitude      : -61.224247
  RH100 (canopy) : 1727.00 m
  Elevation      : 35.67 m

Shot 124160200200076115:
  Latitude       : -0.079945
  Longitude      : -61.223950
  RH100 (canopy) : 2070.00 m
  Elevation      : 36.50 m

Shot 124160200200076116:
  Latitude       : -0.079523
  Longitude      : -61.223653
  RH100 (canopy) : 1712.00 m
  Elevation      : 36.80 m

Shot 124160200200076117:
  Latitude       : -0.079101
  Longitude      : -61.223356
  RH100 (canopy) : 1876.00 m
  Elevation      : 32.94 m

Shot 124160200200076118:
  Latitude       : -0.078680
  Longitude      : -61.223059
  RH100 (canopy) : 1594.00 m
  Elevation      : 33.81 m



In [ ]:
import h5py
import numpy as np
import csv
from geopy.distance import geodesic


def extract_good_gedi_shots(h5_file_path, beam_name="BEAM0000"):
    """Extracts high-quality GEDI shots from a given beam."""
    f = h5py.File(files[0], "r")
    beam = f[beam_name]

    quality = beam['l2b_quality_flag'][:]
    good = quality == 1

    lat = beam['geolocation/lat_lowestmode'][:][good]
    lon = beam['geolocation/lon_lowestmode'][:][good]
    rh100 = beam['rh100'][:][good]
    elev = beam['geolocation/elev_lowestmode'][:][good]
    shot_id = beam['shot_number'][:][good]

    return list(zip(shot_id, lat, lon, rh100, elev))


def get_gedi_near_cluster(cluster_lat, cluster_lon, gedi_shots, radius_m=100):
    """Filters GEDI shots within `radius_m` meters of a given cluster coordinate."""
    nearby = []
    for shot in gedi_shots:
        _, lat, lon, rh, elev = shot
        d = geodesic((lat, lon), (cluster_lat, cluster_lon)).meters
        if d <= radius_m:
            nearby.append((lat, lon, rh, elev))
    return nearby


def analyze_cluster_with_gedi(cluster_lat, cluster_lon, gedi_shots):
    """Analyzes GEDI data near a cluster and determines if it is a potential anomaly."""
    nearby = get_gedi_near_cluster(cluster_lat, cluster_lon, gedi_shots)
    if len(nearby) < 3:
        return {"valid": False, "reason": "Not enough nearby GEDI shots"}

    rh_values = [pt[2] for pt in nearby]
    elev_values = [pt[3] for pt in nearby]

    avg_rh = np.mean(rh_values)
    elev_std = np.std(elev_values)

    is_valid = avg_rh < 10 and elev_std < 1.0
    return {
        "valid": is_valid,
        "avg_rh": avg_rh,
        "elev_std": elev_std,
        "num_shots": len(nearby)
    }


# === Example Usage ===
    # 1. Extract good GEDI shots from file
h5_file = "data/2025-06-07-cf550f/GEDI02_B_2021076201205_O12812_01_T09928_02_003_01_V002.h5"  # replace with actual file
gedi_shots = extract_good_gedi_shots(h5_file, beam_name="BEAM0000")

# 2. Define your anomaly cluster coordinates
cluster_lat = cluster_coord[50][0]
cluster_lon = cluster_coord[50][1]

# 3. Analyze
result = analyze_cluster_with_gedi(latitude, longitude, gedi_shots)

# 4. Show results
if result['valid']:
    print("✅ Valid anomaly!")
    print(f"  RH100 avg: {result['avg_rh']:.2f} m")
    print(f"  Elevation std dev: {result['elev_std']:.2f} m")
    print(f"  Shots used: {result['num_shots']}")
else:
    print("❌ Not valid:", result.get("reason", "Does not meet criteria"))

❌ Not valid: Not enough nearby GEDI shots


In [5]:
!pip install sentinelhub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
len(cluster_coord)

In [6]:
boxes = []
for i in range(len(cluster_coord)):
  latitude = cluster_coord[i][0]
  longitude = cluster_coord[i][1]
  anamolies = cluster_coord[i][2]
  print(latitude,longitude)
  # Step 3: Create a bounding box: [west, south, east, north]
  buffer_deg = 0.005  # ~200 meters
  bbox = [
      longitude - buffer_deg,  # west
      latitude - buffer_deg,   # south
      longitude + buffer_deg,  # east
      latitude + buffer_deg    # north
  ]
  boxes.append(((latitude, longitude),bbox,anamolies))

-3.12217361111111 -52.16988368055558
-3.2758108108108117 -53.6978299549549
-5.662128787878787 -51.38134848484849
-5.871087962962962 -51.6009274691358
-5.8854816666666645 -51.04386166666668
-5.894903735632186 -51.041958333333326
5.455838235294117 -52.99803186274511
4.898286616161615 -52.353503787878786
3.8228446969696974 -51.094462121212125
3.6335257936507936 -51.093343253968236
-0.7262556306306306 -48.04488851351353
-0.8238234126984126 -48.70275198412699
-0.845197463768116 -48.10682427536232
-0.9688496376811593 -48.251585144927546
-1.003065 -48.899011666666674
-1.0045649224806201 -48.904020348837186
-1.1017286036036034 -48.16831193693693
-1.1969450000000001 -48.08788166666668
-1.197194047619047 -48.094017857142866
-1.300732142857143 -48.33708690476191
-1.3145749999999998 -48.24209166666666
-1.6397514367816086 -48.832435344827594
-2.65265036231884 -48.45057910628019
-2.837734375 -48.92954687499999
-2.9359583333333337 -48.87826282051282
-2.9388750000000003 -48.87525543478261
-2.953784420

In [8]:
import numpy as np
from scipy.ndimage import gaussian_gradient_magnitude, maximum_filter, label

def calculate_features(elevation_array):
    # Basic statistics
    mean_elev = np.mean(elevation_array)
    var_elev = np.var(elevation_array)
    max_elev = np.max(elevation_array)
    min_elev = np.min(elevation_array)
    range_elev = max_elev - min_elev

    # Slope approx: gradient magnitude (smoothed)
    slope = gaussian_gradient_magnitude(elevation_array, sigma=1)
    mean_slope = np.mean(slope)
    max_slope = np.max(slope)

    # Roughness = std deviation of slope
    roughness = np.std(slope)

    # Identify local peaks (simple way: pixels that are max in their 3x3 neighborhood)
    neighborhood_size = 3
    local_max = (elevation_array == maximum_filter(elevation_array, size=neighborhood_size))
    num_local_peaks = np.sum(local_max)

    # Flat area: pixels within 5% of mean elevation considered flat
    flat_thresh = 0.05 * mean_elev
    flat_area_percent = np.sum(np.abs(elevation_array - mean_elev) < flat_thresh) / elevation_array.size * 100

    # Output dictionary
    features = {
        'mean_elevation': mean_elev,
        'variance_elevation': var_elev,
        'max_elevation': max_elev,
        'min_elevation': min_elev,
        'range_elevation': range_elev,
        'mean_slope': mean_slope,
        'max_slope': max_slope,
        'roughness': roughness,
        'num_local_peaks': num_local_peaks,
        'flat_area_percent': flat_area_percent
    }
    return features

In [ ]:
!pip install sentinelhub

In [9]:
from sentinelhub import SHConfig
from sentinelhub import SHConfig, SentinelHubRequest, MimeType, CRS, BBox, bbox_to_dimensions, DataCollection
from rasterio.transform import rowcol

config = SHConfig()
config.sh_client_id = "client-secret"
config.sh_client_secret = "secret"

final_clusters = []
for clusters in boxes:
  # ✅ Step 2: Set bounding box and resolution
  coord, bbox,anamolies = clusters
  latitude, longitude = coord
  bbox = BBox(bbox=bbox, crs=CRS.WGS84)
  size = bbox_to_dimensions(bbox, resolution=30)

  # ✅ Step 3: Build the DEM request
  request = SentinelHubRequest(
      evalscript='''//VERSION=3
          return [DEM];
      ''',
      input_data=[
          SentinelHubRequest.input_data(
              data_collection=DataCollection.DEM_COPERNICUS_30
          )
      ],
      responses=[SentinelHubRequest.output_response('default', MimeType.TIFF)],
      bbox=bbox,
      size=size,
      data_folder='dem_tiles',
      config=config  # ✅ must include
  )

  # ✅ Step 4: Download the DEM
  dem_data = request.get_data(save_data=True)
  with rasterio.open('dem_tiles/'+request.get_filename_list()[0]) as src:
    elevation = src.read(1)
    transform = src.transform
    crs = src.crs
  row, col = rowcol(transform, longitude, latitude)
  print(row,col)
  cluster_elev = elevation[row, col]
  print("Elevation at cluster:", cluster_elev, "meters")
  patch = elevation[row-10:row+10, col-10:col+10]
  patch_mean = np.mean(patch)
  patch_std = np.std(patch)
  if patch_std>0:
    final_clusters.append(((latitude,longitude), anamolies,calculate_features(elevation)))
  print(f"Anomaly elev: 255 m, Surrounding avg: {patch_mean:.2f} m, std dev: {patch_std:.2f} m")

18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding avg: 255.00 m, std dev: 0.00 m
18 18
Elevation at cluster: 255 meters
Anomaly elev: 255 m, Surrounding a

In [10]:
final_clusters

[((np.float64(3.6335257936507936), np.float64(-51.093343253968236)),
  21,
  {'mean_elevation': np.float64(219.4835646457268),
   'variance_elevation': np.float64(7774.040818264812),
   'max_elevation': np.uint8(255),
   'min_elevation': np.uint8(0),
   'range_elevation': np.uint8(255),
   'mean_slope': np.float64(1.6493791088385683),
   'max_slope': np.uint8(15),
   'roughness': np.float64(4.068595719231238),
   'num_local_peaks': np.int64(1319),
   'flat_area_percent': np.float64(0.0)}),
 ((np.float64(-0.845197463768116), np.float64(-48.10682427536232)),
  23,
  {'mean_elevation': np.float64(247.73557341124908),
   'variance_elevation': np.float64(1799.656886468132),
   'max_elevation': np.uint8(255),
   'min_elevation': np.uint8(0),
   'range_elevation': np.uint8(255),
   'mean_slope': np.float64(1.3528122717311906),
   'max_slope': np.uint8(15),
   'roughness': np.float64(3.680103818440784),
   'num_local_peaks': np.int64(1331),
   'flat_area_percent': np.float64(97.15120525931337)

In [12]:
def create_prompt(final_clusters):
    prompt = (
        "I have data for 33 geographic regions, each defined by a (37x37) elevation patch.\n"
        "For each region, we have:\n"
        "- Geographic location (latitude, longitude)\n"
        "- Number of detected anomalies (possible man-made structures or irregularities)\n"
        "- Elevation-based terrain features:\n"
        "    • Mean elevation (meters)\n"
        "    • Elevation variance\n"
        "    • Maximum elevation\n"
        "    • Minimum elevation\n"
        "    • Elevation range\n"
        "    • Mean slope (approximate gradient magnitude)\n"
        "    • Maximum slope\n"
        "    • Terrain roughness (std deviation of slope)\n"
        "    • Number of local peaks (possible mounds)\n"
        "    • Percentage of flat area (pixels close to mean elevation)\n"
        "    • Remoteness classification (deep forest, moderately remote, near settlement)\n\n"
        "Here is the data for 33 regions:\n"
    )

    for i, ((lat, lon), num_anomalies, features) in enumerate(final_clusters, start=1):
        # Infer remoteness qualitatively based on lat/lon
        remoteness = "Deep Amazon rainforest (very remote)"
        if abs(lat) < 1.5 and abs(lon + 48) < 3:
            remoteness = "Near civilization or infrastructure"
        elif abs(lat - 3.5) < 1.5 and abs(lon + 51) < 3:
            remoteness = "Moderately remote"

        prompt += (
            f"Region {i}:\n"
            f"  Location: ({lat:.5f}, {lon:.5f})\n"
            f"  Number of anomalies: {num_anomalies}\n"
            f"  Terrain features:\n"
            f"    - Mean elevation: {features['mean_elevation']:.2f} m\n"
            f"    - Elevation variance: {features['variance_elevation']:.2f}\n"
            f"    - Max elevation: {features['max_elevation']:.2f} m\n"
            f"    - Min elevation: {features['min_elevation']:.2f} m\n"
            f"    - Elevation range: {features['range_elevation']:.2f} m\n"
            f"    - Mean slope: {features['mean_slope']:.2f}\n"
            f"    - Max slope: {features['max_slope']:.2f}\n"
            f"    - Roughness: {features['roughness']:.2f}\n"
            f"    - Number of local peaks: {features['num_local_peaks']}\n"
            f"    - Flat area percentage: {features['flat_area_percent']:.2f}%\n"
            f"    - Remoteness: {remoteness}\n\n"
        )

    prompt += (
        "Please analyze these 33 regions and identify the **top 5** regions that are most likely "
        "to indicate the presence of **ancient civilizations or archaeological remains**.\n\n"
        "Your reasoning should take into account:\n"
        "- Higher number of anomalies\n"
        "- Moderate elevation relief (not extremely flat or too steep)\n"
        "- Signs of elevated flat terrain (possible platforms)\n"
        "- Local elevation peaks (possible mounds or pyramids)\n"
        "- Terrain roughness suggestive of man-made disruption\n"
        "- Greater remoteness from modern settlements and infrastructure\n\n"
        "List the top 5 regions with a short explanation for each choice."
    )

    return prompt


In [13]:
prompt = create_prompt(final_clusters)
prompt

'I have data for 33 geographic regions, each defined by a (37x37) elevation patch.\nFor each region, we have:\n- Geographic location (latitude, longitude)\n- Number of detected anomalies (possible man-made structures or irregularities)\n- Elevation-based terrain features:\n    • Mean elevation (meters)\n    • Elevation variance\n    • Maximum elevation\n    • Minimum elevation\n    • Elevation range\n    • Mean slope (approximate gradient magnitude)\n    • Maximum slope\n    • Terrain roughness (std deviation of slope)\n    • Number of local peaks (possible mounds)\n    • Percentage of flat area (pixels close to mean elevation)\n    • Remoteness classification (deep forest, moderately remote, near settlement)\n\nHere is the data for 33 regions:\nRegion 1:\n  Location: (3.63353, -51.09334)\n  Number of anomalies: 21\n  Terrain features:\n    - Mean elevation: 219.48 m\n    - Elevation variance: 7774.04\n    - Max elevation: 255.00 m\n    - Min elevation: 0.00 m\n    - Elevation range: 2

In [14]:
import openai

openai.api_key = 'client-secret'

# Prepare prompt with your data
prompt = create_prompt(final_clusters)

response = openai.chat.completions.create(
    model="gpt-3.5-turbo",  # Replace with the exact GPT-4.1 model name if available
    messages=[
        {"role": "system", "content": "You are an expert archaeological analyst."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=1000,
    temperature=0.7,
)

print(response.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}